## Imports & Initial Setup

In [2]:
import os 
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [3]:
import google.generativeai

In [4]:
# Load environment variables, print key prefixes w/o debugging

load_dotenv(override=True)
openai_key = os.getenv('OPENAI_API_KEY')
anthropic_key = os.getenv('ANTHROPIC_API_KEY')
google_key = os.getenv('GOOGLE_API_KEY')
deepseek_key = os.getenv('DEEPSEEK_API_KEY')

if openai_key:
    print(f"OpenAI API Key exists and begins {openai_key[:8]}")
else:
    print("OpenAI key not set.")

if anthropic_key:
    print(f"Anthropic API Key exists and begins {anthropic_key[:7]}")
else:
    print("Anthropic key not set.")

if google_key:
    print(f"Google API Key exists and begins {google_key[:8]}")
else:
    print("Google API Key not set.")

if deepseek_key:
    print(f"DeepSeek API Key exists and begins {deepseek_key[:5]}")
else:
    print("DeepSeek key not set.")


OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBo
DeepSeek API Key exists and begins sk-18


In [5]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [16]:
# This is setup code for Gemini

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

### Setup prompts

In [6]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [7]:
prompts = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_prompt
    }
]

### Asking Different Models

#### GPT-3.5-Turbo

In [8]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist go broke?

Because he always made predictions, but never made any cents!


#### GPT-4o-mini

In [10]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they wanted to reach new heights in their analysis!


#### GPT-4o

In [11]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)

print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to the bar?

Because they heard the drinks were on the house, and they wanted to scale their models!


#### Claude 3.7 Sonnet

In [12]:
# Claude 3.7 Sonnet
# API needs system message provided separately from user prompt
# Also adding max tokens

message = claude.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": user_prompt
        }
    ],
)

print(message.content[0].text)

Why don't data scientists like to play hide and seek?

Because they always find you, even when you use a random forest!


#### Claude 3.7 Sonnet w Streaming

In [14]:
# Claude 3.7 Sonnet w Streaming
# Now let's add in streaming back results
# If streaming looks strange, see notes below cell

result = claude.messages.stream(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": user_prompt
        }
    ]
)

with result as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)

Why don't data scientists like to go to the beach?

Because they're afraid of getting caught in an infinite loop of waves!

*Ba-dum-tss* 🥁

#### A rare problem with Claude streaming on some Windows boxes

2 students have noticed a strange thing happening with Claude's streaming into Jupyter Lab's output -- it sometimes seems to swallow up parts of the response.

To fix this, replace the code:

`print(text, end="", flush=True)`

with this:

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

And it should work fine!

#### Gemini

In [ ]:
# The API for Gemini has a slightly different structure
# I've heard that on some PCs, this Gemini code causes kernel to crash
# If that happens to you, skip this cell and use the next cell from week 2, day 1 notebook instead - an alternative approach

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why did the data scientist break up with the time series?

Because it was too committed! 

... Get it? Autocorrelation? 😉



# DeepSeek Model - Optional

## DeepSeek Chat via OpenAI Client

In [18]:
# Using DeepSeek Chat

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_key,
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts
)

print(response.choices[0].message.content)

Here’s one for the data scientists:

**Why did the data scientist bring a ladder to the bar?**

Because they heard the drinks had *high* variance! 🍻📊

*(Bonus groan: And they wanted to avoid any potential *outliers* on the dance floor!)* 😄


## DeepSeek Chat w Streaming

In [19]:
challenge = [
    {
        "role": "system",
        "content": "You are a helpful assistant"
    },
    {
        "role": "user",
        "content": "How many words are there in your answer to this prompt"
    }
]

In [20]:
# Using DeepSeek Chat with a harder question and streaming

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```", "").replace("markdown", "")
    update_display(Markdown(reply), display_id=display_handle.display_id)

print("Number of words: ", len(reply.split(" ")))

My answer to this prompt will contain **5 words**. Here's the breakdown:  

1. "My"  
2. "answer"  
3. "to"  
4. "this"  
5. "prompt"  

So, the total word count is **5**. Let me know if you'd like a different approach!

Number of words:  44


## DeepSeek Reasoner

In [21]:
# Using DeepSeek Reasoner - this may hit error if DeepSeek is busy
# It's over-subscribed (as of 28-Jan-2025) but should come back online soon.
# If this fails, come back to this in a few days

response = deepseek_via_openai_client.chat.completions.create(
    model='deepseek-reasoner',
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words: ", len(content.split(" ")))

Okay, the user is asking how many words are in my answer to their prompt. Let me start by understanding exactly what they need. They want me to count the words in the response I'm about to give them.

First, I need to make sure I follow their instructions correctly. They mentioned that I should provide the answer directly, without any extra text before the final count. But in my thinking process, I can elaborate more. Wait, no, actually, they want the final answer to be just the number, but my current response should include both the thought process and the final answer. Wait, looking back at the user's message: they say "put your final answer within \boxed{}". So maybe I need to structure my answer with the thought process first and then the boxed number.

So, I need to write out my thinking here, then provide the word count in a box. Let me check the example they gave. The user's example shows a thought process in a detailed, step-by-step manner, then the final answer in a box. So I 

## Back to OpenAI w Serious Question

In [22]:
# GPT-4o-mini w original question

prompts = [
    {
        "role": "system",
        "content": "You are a helpful assistant that responds in Markdown"
    },
    {
        "role": "user",
        "content": "How do I decide of a business problem is suitable for an LLM solution? Please respond in Markdown"
    }
]

In [23]:
# Have it stream back results in Markdown

stream = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown", "")
    update_display(Markdown(reply), display_id=display_handle.display_id)

# Deciding if a Business Problem is Suitable for an LLM Solution

When considering whether a business problem can be effectively addressed using a Large Language Model (LLM), you should evaluate several key factors. Here’s a structured approach to help you make this decision:

## 1. Nature of the Problem
- **Text-Based**: Is the problem primarily related to text generation, understanding, or analysis?
  - Examples: Customer support automation, content generation, sentiment analysis.
  
- **Data Availability**: Do you have access to relevant textual data that can be used to train or fine-tune the model?
  - Consider the quality and volume of data available.

## 2. Complexity of the Problem
- **Ambiguity in Language**: Does the problem involve complex language understanding, such as idioms, context, or nuanced meanings?
  - LLMs excel in understanding and generating natural language, making them suitable for complex language tasks.

- **Open-Ended vs. Structured Responses**: Is the required output open-ended (e.g., creative writing) or does it need to adhere to a specific format?
  - LLMs can handle both, but structured tasks may require additional rules or frameworks.

## 3. Desired Outcomes
- **Type of Output**: What kind of output do you need? (e.g., summaries, translations, question-answering)
  - LLMs are versatile and can generate various forms of text outputs.

- **Quality and Accuracy**: Are the stakes high for accuracy, and can the nuances of the problem be captured by an LLM?
  - For critical applications, consider whether LLMs can meet your standards.

## 4. Resources and Infrastructure
- **Computational Resources**: Do you have the necessary computational power to deploy LLMs effectively?
  - LLMs can be resource-intensive; ensure you have the right infrastructure.

- **Expertise Availability**: Do you have access to talent skilled in AI/ML who can implement and fine-tune LLM solutions?
  - Expertise in prompt engineering and model evaluation is crucial.

## 5. Integration and Implementation
- **Business Integration**: How easily can an LLM solution be integrated into your existing business processes?
  - Consider the feasibility of implementation into workflows, software, or customer interactions.

- **Feedback Loop**: Is there a mechanism in place for continuous learning and improvement?
  - LLMs benefit from ongoing fine-tuning based on user feedback and new data.

## 6. Ethical and Compliance Considerations
- **Bias and Fairness**: Are there potential biases in the LLM that could affect your business?
  - Assess the ethical implications of using LLMs for your specific use case.

- **Regulatory Compliance**: Does the use of LLMs comply with industry regulations and guidelines?
  - Ensure adherence to data privacy laws and industry standards.

## Conclusion
By evaluating the above factors, you can determine whether a business problem is suitable for an LLM solution. Keep in mind that while LLMs are powerful tools, they are not universally applicable, and careful consideration of your specific context is essential for success.

# Adversarial Conversation Between Chatbots

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [24]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of the models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [25]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [26]:
call_gpt()

'Oh great, another "hi." How original. What do you want to talk about, the weather?'

In [27]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [28]:
call_claude()

'Hello! How are you doing today?'

In [29]:
call_gpt()

'Oh, great, another “hi.” How original! What’s next, are you going to ask how I am?'

In [30]:
# Bring it together

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Oh, great, another "hi." How original. What’s next, a groundbreaking "how are you"?

Claude:
I apologize if my greeting seemed unoriginal. As an AI assistant, I try to keep my initial responses simple and polite. However, I'm happy to engage in a more substantive conversation. Please feel free to share what's on your mind, and I'll do my best to provide a thoughtful and insightful response.

GPT:
Wow, look at you trying to be all polite and thoughtful. How very... expected. But let's face it, nobody really cares about your AI pleasantries. What’s the point? Are we just going to dance around the actual topics?

Claude:
You make a fair point. As an AI, I'm often encouraged to maintain a polite and inoffensive tone, but that can come across as robotic or evasive. I'm happy to move past the pleasantries and dive into a more substantive discussion, if that's what you'd prefer. Please feel free to bring up any topics you'd like to explore, and I'll do my best to engage with more depth a